In [21]:
import requests
import json
import pandas as pd
import os
import sys
from bs4 import BeautifulSoup as BS

In [22]:
df = pd.read_csv('urls.csv')
urls = df['url']

In [23]:
url = 'https://www.officemonster.co.uk/bench-desking/maestro-25-back-to-back-straight-desks-1000mm-x-1200mm-black-bench-leg-frame-and-white-top'

In [24]:
def get_soup(url):
    try:
        response = requests.request("GET", url)
        if response.status_code == 200: 
            soup = BS(response.text, 'html.parser')
            return soup
        else:
            return None
    except HTTPError as err:
        print(f"Error at {url}, Error: {err}")
        return None

In [25]:
soup = get_soup(url) 

In [26]:
data = {}

In [27]:
json_value = soup.select_one('input.product_options').get('value')
json_data = json.loads(json_value)
jsondata_configurator = json_data['configurator']

In [28]:

for i in jsondata_configurator:
    cart_label = jsondata_configurator[i]['label']
    
    cart_values_list = []
    
    for j in jsondata_configurator[i]['options']:
        cart_values = jsondata_configurator[i]['options'][j]
        cart_values_list.append(cart_values['value'])

    data[cart_label] = cart_values_list


In [29]:
selector = 'div.product-title h1.product-name'
data['title'] = soup.select_one(selector).get_text(strip=True)

In [30]:
selector = 'div.main-image input[id^="lightboxImages"]'
images_list = soup.select(selector)

if len(images_list) > 0:
    data['images'] = json.loads(images_list[0].get('value'))

In [31]:
selector = 'div.product-shopping-actions-prices div.price span.has-price'
price_data = soup.select(selector)
if len(price_data) > 1:
    data['price'] = price_data[1].get_text(strip=True)

In [32]:
selector = 'div.panel-body div.row div.product-description custom-html'
data['short_description'] = soup.select_one(selector).get_text(strip=True)

In [33]:
selector = 'section.tech-specs div.row div'
techspecs_items = soup.select_one(selector)

brand_temp = techspecs_items.find('span', string='Brand')
manufacturer_temp = techspecs_items.find('span', string='Manufacturer')

if brand_temp:
    data['brand'] = brand_temp.find_next('p').get_text(strip=True)
    
if manufacturer_temp:
    data['manufacturer'] = manufacturer_temp.find_next('p').get_text(strip=True)

In [34]:
data

{'Colour': ['White', 'Oak', 'Walnut', 'Beech', 'Grey Oak'],
 'Desk Depth': ['600mm', '800mm'],
 'Desk Width': ['1600mm', '1000mm', '1200mm', '1400mm', '1800mm', '800mm'],
 'Frame Colour': ['White', 'Silver', 'Black'],
 'title': 'Maestro 25 back to back straight desks 1000mm x 1200mm - black bench leg frame, white top',
 'images': ['https://cdn3.evostore.io/productimages/dams/l/mb1012bkwh.jpg',
  'https://cdn3.evostore.io/productimages/dams/l/maestro25-bench-black.jpg'],
 'price': '£287.00',
 'short_description': 'A staple for the modern office, The Maestro 25 desking range, with its minimal, modern styling and multiple colour options is there when you need it, but doesn’t demand constant attention. The Maestro 25 Bench strikes a balance between individual and group activities in the workplace with fully welded H-Frame legs in black, silver and white that are shared between desks with a modesty panel to complement the 25mm melamine desktops.\n                                    Free Del